In [1]:
import pandas as pd
# import fireducks.pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
from pandas.api.types import is_numeric_dtype
from pymongo import MongoClient
import json

import polars as pl
from typing import List, Dict, Union, Any
import os

import warnings
warnings.filterwarnings('ignore')

In [22]:
import sys
import pathlib
sys.path.append('/home/sebasav/Documentos/FACSAT-2/Git_Repo/DataMiningFACSAT2/airflow')

In [43]:
from dags.DAG_ETL import run_etl_process

In [44]:
run_etl_process(dictionary_path="../notebooks/outputs/diccionario_de_datos.csv")

2024-11-17 20:55:26,153 - dags.DAG_ETL - INFO - Processing Node OBC: 1, Table # 4 - run_etl_process:412
2024-11-17 20:55:26,157 - dags.DAG_ETL - INFO - Creating MongoDB configuration - extract_data:66
2024-11-17 20:55:26,157 - dags.DAG_ETL - INFO - Initializing MongoDB configuration - __init__:24
2024-11-17 20:55:26,158 - dags.DAG_ETL - INFO - Establishing MongoDB connection - extract_data:70
2024-11-17 20:55:26,159 - dags.DAG_ETL - INFO - Verifying MongoDB connection - extract_data:80
2024-11-17 20:55:31,177 - dags.DAG_ETL - ERROR - Data extraction failed with error: mongodb_secondary:27017: [Errno -3] Temporary failure in name resolution (configured timeouts: socketTimeoutMS: 30000.0ms, connectTimeoutMS: 30000.0ms), Timeout: 5.0s, Topology Description: <TopologyDescription id: 673a9e8e3d030cdcca104de4, topology_type: Single, servers: [<ServerDescription ('mongodb_secondary', 27017) server_type: Unknown, rtt: None, error=AutoReconnect('mongodb_secondary:27017: [Errno -3] Temporary fai

ServerSelectionTimeoutError: mongodb_secondary:27017: [Errno -3] Temporary failure in name resolution (configured timeouts: socketTimeoutMS: 30000.0ms, connectTimeoutMS: 30000.0ms), Timeout: 5.0s, Topology Description: <TopologyDescription id: 673a9e8e3d030cdcca104de4, topology_type: Single, servers: [<ServerDescription ('mongodb_secondary', 27017) server_type: Unknown, rtt: None, error=AutoReconnect('mongodb_secondary:27017: [Errno -3] Temporary failure in name resolution (configured timeouts: socketTimeoutMS: 30000.0ms, connectTimeoutMS: 30000.0ms)')>]>

# Proceso de ETL

## Definicion de funciones

In [2]:
def query_mongodb(client, collection: str = '', query_json: dict = {}, query_project: dict = {}, skip: int = 0, limit: int = 0):
    # Define MongoDB parameters
    mongo_srv_uri = 'mongodb://localhost:27017/'
    mongo_user = ''
    mongo_password = ''
    mongo_db_name = 'facsat2'
    mongo_collection_name = collection

    try:
        # Access the MongoDB database
        db = client[mongo_db_name]
        collection = db[mongo_collection_name]

        # Perform MongoDB operations
        result = collection.find(query_json, query_project).skip(skip).limit(limit) 
    except Exception as e:
        print("No fue posible realizar la consulta\nError:\n")
        print(e)
    
    data = list(result) # pl.DataFrame(list(result))
    return data

def crear_dataframe_polars(
    data: List[Dict[str, Any]], 
    system: str, 
    table: str
) -> pl.DataFrame:
    """
    Creates a DataFrame from JSON telemetry data using Polars for improved performance.
    
    Parameters:
    -----------
    data : List[Dict[str, Any]]
        JSON data containing telemetry information
    system : str
        System identifier to be used in column names
    table : str
        Table identifier to be used in column names
    
    Returns:
    --------
    polars.DataFrame
        Processed DataFrame with restructured telemetry data
        
    Raises:
    -------
    ValueError
        If input data is empty or required columns are missing
    """
    try:
        # Validate input data
        if not data:
            raise ValueError("Input data is empty")
        
        # Create Polars DataFrame from JSON
        # df = pl.DataFrame(data)
        # Normalizar los datos JSON
        df = pl.json_normalize(data)
        
        if df.is_empty():
            raise ValueError("DataFrame is empty")
            
        # Validate required columns
        required_columns = {'Ts', 'Val', 'Param.Name', 'Param.Table'}
        missing_columns = required_columns - set(df.columns)
        if missing_columns:
            raise ValueError(f"Missing required columns: {missing_columns}")
            
        # Convert timestamp to datetime
        df = df.with_columns([
            pl.col('Ts').cast(pl.Int64).map_elements(datetime.fromtimestamp).alias('Date')
        ])
        
        # Initialize final DataFrame schema
        has_index = 'Param.Index' in df.columns
        
        if has_index:
            # Handle indexed parameters
            df = df.with_columns([
                pl.when(pl.col('Param.Index').is_not_null())
                .then(pl.col('Param.Index') + 1)
                .otherwise(pl.lit(0))
                .alias('Param.Index')
                .cast(pl.Int64)
            ])
            
            # Pivot the data
            final_df = (
                df.pivot(
                    values='Val',
                    index='Date',
                    columns=['Param.Name', 'Param.Index'],
                    aggregate_function='first'
                )
            )
            
            # Get column names (excluding 'Date')
            old_columns = [col for col in final_df.columns if col != 'Date']
            
            # Create new column names
            new_columns = {}
            for col in old_columns:
                # Parse the string representation of the tuple
                # Format is like '{"mag",1}' or '{"mag",0}'
                parts = col.strip('{}').split(',')
                param_name = parts[0].strip('"')  # Remove quotes
                index = int(parts[1].strip())     # Convert to int
                new_name = (
                    f"{system}{str(table)}_{param_name}"
                    if index == 0 else
                    f"{system}{str(table)}_{param_name}_{str(int(index))}"
                )
                new_columns[col] = new_name
            
        else:
            # Handle non-indexed parameters
            final_df = (
                df.pivot(
                    values='Val',
                    index='Date',
                    columns='Param.Name',
                    aggregate_function='first'
                )
            )
            
            # Rename columns with system and table identifiers
            new_columns = {
                col: f"{system}{str(table)}_{col}"
                for col in final_df.columns
                if col != 'Date'
            }
        
        # Apply column renaming
        final_df = final_df.rename(new_columns)
        
        return final_df
    
    except Exception as e:
        error_msg = f"Error processing telemetry data: {str(e)}"
        print(error_msg)
        raise RuntimeError(error_msg) from e

# def check_columns_missmatch(data_frame):
#     # Get a complete set of all unique columns across all dataframes
#     all_columns = set()
#     for df in data_frame:
#         all_columns.update(df.columns)

#     # For each DataFrame, add missing columns with null values before concatenation
#     aligned_dfs = []
#     for df in data_frame:
#         missing_cols = list(all_columns - set(df.columns))
        
#         # Create a new DataFrame with missing columns filled with nulls
#         if missing_cols:
#             # Create a DataFrame with null values for missing columns
#             null_df = pl.DataFrame({
#                 col: [None] * len(df) for col in missing_cols
#             })
#             # Combine with original DataFrame
#             aligned_df = pl.concat([df, null_df], how='horizontal')
#         else:
#             aligned_df = df
        
#         aligned_dfs.append(aligned_df)

#     return aligned_dfs

def check_columns_missmatch(dataframes: List[pl.DataFrame]) -> List[pl.DataFrame]:
    """
    Optimizada versión de la función para alinear columnas y tipos de datos en DataFrames de Polars.
    
    Args:
        dataframes: Lista de DataFrames de Polars a alinear
        
    Returns:
        Lista de DataFrames alineados con tipos de datos consistentes
    """
    if not dataframes:
        return []
    
    # Obtener schema de todas las columnas en un solo paso
    schemas = {
        col: dtype
        for df in dataframes
        for col, dtype in df.schema.items()
    }
    
    # Determinar el tipo más compatible para cada columna
    final_schema = {}
    for col in schemas:
        types = [df.schema.get(col) for df in dataframes if col in df.columns]
        final_schema[col] = (
            pl.Float64 if pl.Int64 in types and pl.Float64 in types
            else types[0]
        )
    
    # Función helper para procesar un DataFrame
    def align_dataframe(df: pl.DataFrame) -> pl.DataFrame:
        # Preparar expresiones para las columnas existentes y nuevas
        expressions = []
        
        # Procesar todas las columnas necesarias
        for col, target_type in final_schema.items():
            if col in df.columns:
                # Si la columna existe y necesita conversión
                if df.schema[col] != target_type:
                    expressions.append(
                        pl.col(col).cast(target_type).alias(col)
                    )
            else:
                # Si la columna no existe, crear con nulls
                expressions.append(
                    pl.lit(None).cast(target_type).alias(col)
                )
        
        # Si no hay transformaciones necesarias, retornar el DataFrame original
        if not expressions:
            return df
            
        # Aplicar todas las transformaciones en una sola operación
        return df.with_columns(expressions)
    
    # Procesar todos los DataFrames usando map
    return list(map(align_dataframe, dataframes))


def clean_data_with_flexible_references(data_df: pl.DataFrame, ref_df: pl.DataFrame) -> pl.DataFrame:
    """
    Limpia datos utilizando referencias inválidas y válidas de forma flexible.

    Args:
        data_df: DataFrame de datos a limpiar.
        ref_df: DataFrame de referencias con columnas 'Variable', 'Invalido' y 'Valido'.

    Returns:
        DataFrame limpio.
    """

    # def clean_column(col_name, col_refs):
    #     if col_refs.is_empty():  # Check if reference DataFrame is empty
    #         return pl.col(col_name)  # Keep the original column

    #     invalid_value = col_refs.get_column('Invalido').item() if not col_refs.is_empty() else None
    #     valid_values = col_refs.get_column('Valido').item() if not col_refs.is_empty() else None

    #     invalid_condition = pl.col(col_name) == invalid_value if invalid_value is not None else pl.lit(False)
    #     valid_condition = pl.col(col_name).is_in(valid_values) if valid_values is not None else pl.lit(True)

    #     final_condition = ~(invalid_condition | ~valid_condition)

    #     return pl.when(final_condition).then(pl.col(col_name)).otherwise(None).alias(col_name)
    def clean_column(col_name, col_refs):
        if col_refs.is_empty():  # Check if reference DataFrame is empty
            return pl.col(col_name)  # Keep the original column

        invalid_value = col_refs.get_column('Invalido').item() if not col_refs.is_empty() else None
        # valid_values = col_refs.with_columns(pl.col('Valido').str.extract_all(r'(\d+)').list.eval(pl.element().cast(pl.Int64)).alias('list_column')) if not col_refs.is_empty() else None # col_refs.get_column('Valido').to_list()

        # Extract the list of valid values
        if not col_refs.is_empty():
            valid_values = (col_refs.select(
                pl.col('Valido')
                .str.extract_all(r'(\d+)')
                .list.eval(pl.element().cast(pl.Int64))
            ).item())
        else:
            valid_values = None

        invalid_condition = pl.col(col_name) == invalid_value if invalid_value is not None else pl.lit(False)
        valid_condition = pl.col(col_name).is_in(valid_values) if valid_values is not None else pl.lit(True)

        final_condition = ~(invalid_condition | ~valid_condition)

        return pl.when(final_condition).then(pl.col(col_name)).otherwise(None).alias(col_name)

    return data_df.select([
        clean_column(col_name, ref_df.filter(pl.col('Variable') == col_name))
        for col_name in data_df.columns
    ])

def remove_full_null_rows(df: pl.DataFrame) -> pl.DataFrame:
    """
    Remove rows from a Polars DataFrame where all columns (except 'Date') contain null values.
    
    Parameters:
    -----------
    df : pl.DataFrame
        Input Polars DataFrame
        
    Returns:
    --------
    pl.DataFrame
        DataFrame with rows removed where all columns (except 'Date') were null
    """
    # Get all column names except 'Date'
    cols_to_check = [col for col in df.columns if col != 'Date']
    
    # Create a boolean mask for each row where True means the row has at least one non-null value
    # in columns other than 'Date'
    mask = df.select(
        pl.any_horizontal(~pl.col(cols_to_check).is_null())
    ).to_series()
    
    # Filter the DataFrame using the mask
    return df.filter(mask)

def send_to_mongodb(data_df: pl.DataFrame, client, to_database: str, to_collection: str):
    # Convertir el DataFrame a una lista de diccionarios
    data_df = data_df.with_columns(
        pl.lit(datetime.now()).alias("fecha_creacion"),
        pl.lit(None).cast(pl.Datetime).alias("fecha_borrado")
    )

    data = data_df.to_dicts()

    # Conectar a MongoDB
    db = client[to_database]
    collection = db[to_collection]

    # Insertar los datos en MongoDB
    collection.insert_many(data)

def process_mongodb_data(
    mongodb_uri: str = "mongodb://localhost:27017/",
    batch_size: int = 500000,
    dictionary_path: str = "/outputs/diccionario_de_datos.csv",
    output_path: str = "param_data_df.csv"
) -> None:
    """
    Process MongoDB data using Polars for improved performance.
    
    Parameters:
    -----------
    mongodb_uri : str
        MongoDB connection URI
    batch_size : int
        Number of documents to process in each batch
    dictionary_path : str
        Path to the data dictionary CSV file
    output_path : str
        Path where the final CSV will be saved
    """
    try:
        # Connect to MongoDB
        client = MongoClient(mongodb_uri, connectTimeoutMS=30000)

        # Initialize empty DataFrame with Date column
        final_dataframe = pl.DataFrame(schema={"Date": pl.Datetime})
        
        # Read and process dictionary data using Polars
        dic_datos = pl.read_csv(dictionary_path)
        
        # Process indices
        dic_datos = dic_datos.with_columns([
            pl.when(pl.col('Indice') >= 0)
            .then(pl.col('Indice') + 1)
            .otherwise(pl.lit(0))
            .cast(pl.Int64)
            .alias('Indice')
        ])
        
        # Create variable names with indices
        dic_datos = dic_datos.with_columns([
            pl.when(pl.col('Indice') > 0)
            .then(pl.col('Sistema') + pl.col('Tabla').cast(pl.String) + "_" + pl.col('Variable') + "_" + pl.col('Indice').cast(pl.Int64).cast(pl.Utf8))
            .otherwise(pl.col('Sistema') + pl.col('Tabla').cast(pl.String) + "_" + pl.col('Variable'))
            .alias('Variable')
        ])
        
        # Group by Sistema, Nodo, Tabla
        loop_over_nodes = (
            dic_datos.group_by(['Sistema', 'Nodo', 'Tabla'])
            .agg(pl.col('Indice').mean())
            .sort(['Nodo', 'Tabla'])
        )

        # loop_over_nodes = loop_over_nodes.filter(pl.col('Sistema') == "ADCS", pl.col('Nodo') == 4, pl.col('Tabla') == 151)
        
        # Process each node
        for row in loop_over_nodes.iter_rows(named=True):
            sistema = row['Sistema']
            num_nodo = row['Nodo']
            num_tabla = row['Tabla']
            
            print(f"Processing Node {sistema}: {num_nodo}, Table # {num_tabla}")
            
            skip_count = 0
            node_data_frames = []
            combined_node_df = pl.DataFrame()
            
            while True:
                # Query MongoDB in batches
                node_dict = query_mongodb(
                    client=client,
                    collection='ParamData',
                    query_json={"Param.Node": num_nodo, "Param.Table": num_tabla},
                    limit=batch_size,
                    skip=skip_count
                )
                
                if not node_dict:
                    break
                
                # Process batch using Polars
                try:
                    node_df = crear_dataframe_polars(node_dict, sistema, num_tabla)
                    node_data_frames.append(node_df)
                except Exception as e:
                    print(f"Error processing batch for Node {num_nodo}, Table {num_tabla}: {str(e)}")
                    continue
                
                skip_count += batch_size
            
            # Combine all batches for this node
            if node_data_frames:
                # Concatenate all batches for this node
                node_data_frames_checked = check_columns_missmatch(node_data_frames)
                combined_node_df = pl.concat([df.select(node_data_frames_checked[0].columns) for df in node_data_frames_checked])

                # Optimize final dataframe before writing
                if not combined_node_df.is_empty():
                    # Sort by Date
                    combined_node_df = combined_node_df.sort('Date')
                    
                    # Get all column names except 'Date' and 'Tabla'
                    numeric_cols = [col for col in combined_node_df.columns if col not in ['Date', 'Tabla']]

                    # Cast those columns to Float32
                    combined_node_df = combined_node_df.with_columns([
                        pl.col(numeric_cols).cast(pl.Float32)
                    ])

                    cleaned_node_df = clean_data_with_flexible_references(combined_node_df, dic_datos)
                    fully_cleaned_node_df = remove_full_null_rows(cleaned_node_df)
                    
                    # Write to CSV with optimizations
                    # output_path = f"/outputs/{sistema}_{num_tabla}_collection.csv"
                    # fully_cleaned_node_df.write_csv(
                    #     output_path,
                    #     float_precision=2,
                    #     batch_size=batch_size
                    # )

                    send_to_mongodb(fully_cleaned_node_df, client, "etl_data", f"{sistema}_{num_tabla}_collection")

                    print(f"Successfully wrote data to {output_path}")
                    print(f"Final DataFrame shape: {fully_cleaned_node_df.shape}")
                    print(f"Memory usage: {fully_cleaned_node_df.estimated_size() / 1024 / 1024:.2f} MB")
                
    except Exception as e:
        print(f"Error in data processing pipeline: {str(e)}")
        raise
    
    finally:
        # Close MongoDB connection
        client.close()

## Ejecucion del proceso

In [3]:
curr_dir = os.path.abspath(os.getcwd())
target_dict = curr_dir + "/outputs/diccionario_de_datos.csv"
process_mongodb_data(dictionary_path=target_dict)

Processing Node OBC: 1, Table # 4
Successfully wrote data to param_data_df.csv
Final DataFrame shape: (60537, 11)
Memory usage: 2.77 MB
Processing Node OBC: 1, Table # 92
Successfully wrote data to param_data_df.csv
Final DataFrame shape: (63054, 11)
Memory usage: 2.96 MB
Processing Node OBC: 1, Table # 93
Successfully wrote data to param_data_df.csv
Final DataFrame shape: (63056, 11)
Memory usage: 2.96 MB
Processing Node ADCS: 4, Table # 1
Successfully wrote data to param_data_df.csv
Final DataFrame shape: (25120, 6)
Memory usage: 0.67 MB
Processing Node ADCS: 4, Table # 4
Successfully wrote data to param_data_df.csv
Final DataFrame shape: (25120, 12)
Memory usage: 1.25 MB
Processing Node ADCS: 4, Table # 5
Successfully wrote data to param_data_df.csv
Final DataFrame shape: (25120, 3)
Memory usage: 0.38 MB
Processing Node ADCS: 4, Table # 150
Successfully wrote data to param_data_df.csv
Final DataFrame shape: (212295, 124)
Memory usage: 104.34 MB
Processing Node ADCS: 4, Table # 151
S

In [5]:
import polars as pl

def clean_data_with_references(
    data_df: pl.DataFrame,
    ref_df: pl.DataFrame
) -> pl.DataFrame:
    """
    Clean data using both invalid references and list-based valid references without loops.
    
    Parameters:
    -----------
    data_df : pl.DataFrame
        DataFrame containing the variables to clean
    ref_df : pl.DataFrame
        DataFrame containing invalid and valid reference values per variable
        Expected columns: 'variable_name', 'invalid_value', 'valid_value'
    
    Returns:
    --------
    pl.DataFrame
        Cleaned DataFrame with:
        - invalid values replaced by null
        - values not in valid reference lists replaced by null
    """
    # Create expressions for each column based on reference values
    clean_expressions = []
    
    for col_name in data_df.columns:
        # Get reference data for this column
        col_refs = ref_df.filter(pl.col('Variable') == col_name)
        
        # If no references found for this column, keep original
        if col_refs.height == 0:
            clean_expressions.append(pl.col(col_name))
            continue
            
        # Initialize condition as None
        final_condition = None
        
        # Handle invalid values
        invalid_value = col_refs.get_column('Invalido')[0]
        if isinstance(invalid_value, (int, float, str)):
            invalid_condition = pl.col(col_name).cast(pl.Utf8).eq(str(invalid_value))
            final_condition = invalid_condition
        
        # Handle valid values
        valid_values = col_refs.get_column('Valido')[0]
        if valid_values is not None:
            valid_values = valid_values.to_list()

        if isinstance(valid_values, list) and len(valid_values) > 0:
            # Convert all values to strings for comparison
            valid_values_str = [str(v) for v in valid_values]
            valid_condition = ~pl.col(col_name).cast(pl.Utf8).is_in(valid_values_str)
            
            # Combine with existing condition if any
            if final_condition is not None:
                final_condition = final_condition | valid_condition
            else:
                final_condition = valid_condition
        
        # Create the cleaning expression
        if final_condition is not None:
            clean_expr = pl.when(final_condition).then(None).otherwise(pl.col(col_name)).alias(col_name)
            clean_expressions.append(clean_expr)
        else:
            clean_expressions.append(pl.col(col_name))
    
    # Apply all cleaning expressions at once
    return data_df.select(clean_expressions)

# Example usage:
if __name__ == "__main__":
    data = pl.DataFrame({
        'date': ['2024-10-23', '2024-10-24', '2024-10-25', '2024-10-26', '2024-10-27', '2024-10-28', '2024-10-29', '2024-10-30'],
        'col1': [1, 2, -999, 4, -999, 7, 8, 9],
        'col2': [9, 8, 7, 6, 500, 4, 3, 2],
        'col3': [10.0, -888.8, 30.0, 40.0, 50.0, 60.0, 70.0, 80.0]
    })
    
    # Referencias de valores válidos e inválidos
    invalid_ref = pl.DataFrame(
        {
            'Variable': ['col1', 'col2', 'col3'],
            'Invalido': [None, 500, None],
            'Valido': [
                '[4, 2, 1, 7, 8]',
                None,
                '[10.0, 30.0, 40.0, 50.0, 60.0, 70.0]'
            ]
        }
    )
    
    # Clean the data
    cleaned_data = clean_data_with_flexible_references(data, invalid_ref) # clean_data_with_references(data, invalid_ref)
    print("\nCleaned data with both invalid and valid references:")
    print(cleaned_data)


Cleaned data with both invalid and valid references:
shape: (8, 4)
┌────────────┬──────┬──────┬──────┐
│ date       ┆ col1 ┆ col2 ┆ col3 │
│ ---        ┆ ---  ┆ ---  ┆ ---  │
│ str        ┆ i64  ┆ i64  ┆ f64  │
╞════════════╪══════╪══════╪══════╡
│ 2024-10-23 ┆ 1    ┆ 9    ┆ 10.0 │
│ 2024-10-24 ┆ 2    ┆ 8    ┆ null │
│ 2024-10-25 ┆ null ┆ 7    ┆ 30.0 │
│ 2024-10-26 ┆ 4    ┆ 6    ┆ 40.0 │
│ 2024-10-27 ┆ null ┆ null ┆ 50.0 │
│ 2024-10-28 ┆ 7    ┆ 4    ┆ 60.0 │
│ 2024-10-29 ┆ 8    ┆ 3    ┆ 70.0 │
│ 2024-10-30 ┆ null ┆ 2    ┆ null │
└────────────┴──────┴──────┴──────┘


In [ ]:
def clean_data_with_flexible_references(data_df: pl.DataFrame, ref_df: pl.DataFrame) -> pl.DataFrame:
    """
    Limpia datos utilizando referencias inválidas y válidas de forma flexible.

    Args:
        data_df: DataFrame de datos a limpiar.
        ref_df: DataFrame de referencias con columnas 'Variable', 'Invalido' y 'Valido'.

    Returns:
        DataFrame limpio.
    """

    def clean_column(col_name, col_refs):
        if col_refs.is_empty():  # Check if reference DataFrame is empty
            return pl.col(col_name)  # Keep the original column

        invalid_value = col_refs.get_column('Invalido').item() if not col_refs.is_empty() else None
        valid_values = col_refs.get_column('Valido').item() if not col_refs.is_empty() else None

        invalid_condition = pl.col(col_name) == invalid_value if invalid_value is not None else pl.lit(False)
        valid_condition = pl.col(col_name).is_in(valid_values) if valid_values is not None else pl.lit(True)

        final_condition = ~(invalid_condition | ~valid_condition)

        return pl.when(final_condition).then(pl.col(col_name)).otherwise(None).alias(col_name)

    return data_df.select([
        clean_column(col_name, ref_df.filter(pl.col('Variable') == col_name))
        for col_name in data_df.columns
    ])
# Example usage:
if __name__ == "__main__":
    data = pl.DataFrame({
        'date': ['2024-10-23', '2024-10-24', '2024-10-25', '2024-10-26', '2024-10-27', '2024-10-28', '2024-10-29', '2024-10-30'],
        'col1': [1, 2, -999, 4, -999, 7, 8, 9],
        'col2': [9, 8, 7, 6, 500, 4, 3, 2],
        'col3': [10.0, -888.8, 30.0, 40.0, 50.0, 60.0, 70.0, 80.0]
    })
    
    # Referencias de valores válidos e inválidos
    invalid_ref = pl.DataFrame(
        {
            'Variable': ['col1', 'col2', 'col3'],
            'Invalido': [None, 500, None],
            'Valido': [
                [4, 2, 1, 7, 8],
                None,
                [10.0, 30.0, 40.0, 50.0, 60.0, 70.0]
            ]
        }
    )
    
    # Clean the data
    cleaned_data = clean_data_with_flexible_references(data, invalid_ref)
    print("\nCleaned data with both invalid and valid references:")
    print(cleaned_data)


Cleaned data with both invalid and valid references:
shape: (8, 4)
┌────────────┬──────┬──────┬──────┐
│ date       ┆ col1 ┆ col2 ┆ col3 │
│ ---        ┆ ---  ┆ ---  ┆ ---  │
│ str        ┆ i64  ┆ i64  ┆ f64  │
╞════════════╪══════╪══════╪══════╡
│ 2024-10-23 ┆ 1    ┆ 9    ┆ 10.0 │
│ 2024-10-24 ┆ 2    ┆ 8    ┆ null │
│ 2024-10-25 ┆ null ┆ 7    ┆ 30.0 │
│ 2024-10-26 ┆ 4    ┆ 6    ┆ 40.0 │
│ 2024-10-27 ┆ null ┆ null ┆ 50.0 │
│ 2024-10-28 ┆ 7    ┆ 4    ┆ 60.0 │
│ 2024-10-29 ┆ 8    ┆ 3    ┆ 70.0 │
│ 2024-10-30 ┆ null ┆ 2    ┆ null │
└────────────┴──────┴──────┴──────┘


# EDA

## OBC

In [ ]:
obc = query_mongodb('ParamData', {"Param.Node": nodos['OBC'], "Ts": { "$gte": timestamp_mes_atras}}, limit=0)
obc_df = crear_dataframe(obc)

In [ ]:
print(obc_df.duplicated().sum())

In [ ]:
# Identify numerical columns
numerical_columns = obc_df.select_dtypes(include=[np.number]).columns

# Perform univariate analysis on numerical columns
for column in numerical_columns:
    # For continuous variables
    if len(obc_df[column].unique()) > 10:  # Assuming if unique values > 10, consider it continuous
        plt.figure(figsize=(8, 6))
        sns.histplot(obc_df[column], kde=True)
        plt.title(f'Histogram of {column}')
        plt.xlabel(column)
        plt.ylabel('Frequency')
        plt.show()
    else:  # For discrete or ordinal variables
        plt.figure(figsize=(8, 6))
        ax = sns.countplot(x=column, data=obc_df)
        plt.title(f'Count of {column}')
        plt.xlabel(column)
        plt.ylabel('Count')
        
        # Annotate each bar with its count
        for p in ax.patches:
            ax.annotate(format(p.get_height(), '.0f'), 
                        (p.get_x() + p.get_width() / 2., p.get_height()), 
                        ha = 'center', va = 'center', 
                        xytext = (0, 5), 
                        textcoords = 'offset points')
        plt.show()

## Procesamiento y transformacion de los datos

In [ ]:
for col_name in obc_df.columns:
    obc_df[col_name] = obc_df[col_name].astype(int) if np.all(obc_df[col_name] == obc_df[col_name].astype(int)) else obc_df[col_name]

# Otros

In [ ]:
node_1 = query_mongodb('ParamData', {"Param.Node": 1, "Ts": { "$gte": timestamp_mes_atras}}, limit=0)
node_4 = query_mongodb('ParamData', {"Param.Node": 4, "Ts": { "$gte": timestamp_mes_atras}}, limit=0)
node_1_df = crear_dataframe(node_1)
node_4_df = crear_dataframe(node_4)

In [ ]:
result_df = node_1_df.merge(node_4_df, on=['Date'], how='outer')

In [ ]:
result_df_filtered = result_df.filter(regex='Date|_x|_y|_z')

In [ ]:
a = result_df_filtered[['Date', 'temp_mcu_x', 'temp_mcu_y']]

In [ ]:
a_filtered = a.dropna(subset=a.columns)

In [ ]:
a_filtered = a.loc[:, a.std() != 0]

In [ ]:
plt.figure(figsize=(8, 6))
mask = np.triu(np.ones_like(a_filtered.corr(), dtype=np.bool))
sns.heatmap(a_filtered.corr(), annot=False, cmap="coolwarm", mask=mask, vmin=-1, vmax=1)